In [1]:
!pip3 install transformers datasets sentencepiece peft -q
!pip install torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q # Updating torch since we need the latest version
!pip install torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q
!pip uninstall tensorflow -y # If we don't do this, TF will take over TPU and cause permission error for PT
!cp /kaggle/input/utils-xla/spmd_util.py . # From this repo: https://github.com/HeegyuKim/torch-xla-SPMD


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.
torchaudio 2.0.0 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboardx 2.6.1 requires protobuf>=4.22.3, but you h

In [2]:
import os
import pandas as pd
import numpy as np
import datasets
import torch.optim as optim
import torch_xla.debug.profiler as xp
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp # We also import mp modules if we wanna use that for some reason
import torch_xla.distributed.parallel_loader as pl
import torch_xla.test.test_utils as test_utils
import torch
import torch.nn as nn
import re
import torch_xla.experimental.xla_sharding as xs
import torch_xla.core.xla_model as xm
from transformers import (
    GPTNeoXConfig, T5Config, LlamaConfig, AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, AutoConfig, AutoModelForSequenceClassification
) # You can use any of models with those configs (even flan T5 xxl!). Other models are not supported.

from transformers import logging as hf_logging
import torch.nn.functional as F
import torch_xla.runtime as xr

xr.use_spmd()

import torch_xla.experimental.xla_sharding as xs # "experimental" prefix always means you're gonna have a good time LMAO
from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor
from torch_xla.experimental.xla_sharding import Mesh

from peft import LoraConfig, TaskType, get_peft_model # If we wanna use peft. Quantazation requiers GPU though. You'll have to download already quantazed models
from spmd_util import partition_module                # You could experiment with using already quantazed models like 4bit/Llama-2-7b-Chat-GPTQ if you're feeling funny
from datasets import Dataset, load_dataset, concatenate_datasets
from dataclasses import dataclass
from tqdm import tqdm

import transformers
import datasets
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.metrics import roc_auc_score

!export USE_TORCH=True #If we don't do this, transformers will seemingly bork the session upon import. Really weird error.
os.environ["PJRT_DEVICE"] = "TPU"
os.environ.pop('TPU_PROCESS_ADDRESSES')
os.environ.pop('CLOUD_TPU_TASK_ID')
hf_logging.set_verbosity_error() # It can still display warnings which is a bit annoying but whatever


MAX_INPUT=512
MODEL = "/kaggle/input/best-llm-starter-pack/nous_hermes/Nous-Hermes-Llama2-13b/snapshots/8f95aa9cd207db7b24179fc779c2b8973e71bee2" #You should be able to use 13B model with no changes! There should be enough HBM

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_ai = pd.read_csv('/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv')
ai_samples_500 = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv')
persuade_df = pd.read_csv('/kaggle/input/persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv')
df_llama = pd.read_csv('/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_70b_v1.csv')
df_falcon = pd.read_csv('/kaggle/input/daigt-data-llama-70b-and-falcon180b/falcon_180b_v1.csv')
proper_train = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
aug_df = pd.read_csv('/kaggle/input/argugpt/argugpt.csv')
corrected_df = pd.read_csv('/kaggle/input/grammar-corrected-persuade-10k/corrected_persuade_10k.csv')

In [4]:
aug_df = pd.DataFrame({'text': aug_df['text'], 'generated': 1})
proper_train_ai = pd.DataFrame({'text': proper_train['text'][proper_train['label']==1], 'generated': 1}).reset_index(drop=True)
df_nbroad = pd.concat([df_llama, df_falcon]).reset_index(drop=True)
nbroad_ai = pd.DataFrame({'text': df_nbroad['generated_text'], 'generated': 1}).reset_index(drop=True)
ai_ds = pd.concat([proper_train_ai, nbroad_ai, ai_samples_500.drop(columns=['prompt_id', 'id']), aug_df]).reset_index(drop=True)
human_ds = pd.DataFrame({'text': corrected_df['corrected_text'], 'generated': 0}).reset_index(drop=True)
final_df = pd.concat([ai_ds, human_ds]).sample(frac=1).reset_index(drop=True)
final_df

lengths = []
for sample in final_df['text']:
    lengths.append(len(sample.split(' ')))
max(lengths), min(lengths), sum(lengths) / len(lengths)

(1638, 4, 361.42858331946064)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=512, padding='max_length', truncation=True) #We could try `padding_to_multiple_of=128`

In [7]:
ds = Dataset.from_pandas(final_df).rename_column("generated", "label").train_test_split(test_size=0.15)
ds['train'] = ds['train'].map(preprocess_function, batched=False, num_proc=96, remove_columns=['text'])
ds['test'] = ds['test'].map(preprocess_function, batched=False, num_proc=96, remove_columns=['text'])
ds

Map (num_proc=96): 100%|██████████| 20423/20423 [00:27<00:00, 732.86 examples/s]
/usr/local/lib/python3.8/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
Map (num_proc=96): 100%|██████████| 3605/3605 [00:25<00:00, 141.25 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 20423
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3605
    })
})

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2, torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 3/3 [02:56<00:00, 58.70s/it]


In [9]:
FLAGS = {'MAX_INPUT': 512,
         'LOGGING_STEPS': 100,
         'NUM_EPOCHS': 1,
         'BATCH_SIZE': 8, #Making batch_size lower then 8 will result in slower training, but will allow for larger models\context. Fortunately, we have 128GBs. Setting higher batch_size doesn't seem to improve time.
          'NUM_STEPS': len(ds)} 

In [10]:
training_loader = torch.utils.data.DataLoader(ds['train'], batch_size=FLAGS['BATCH_SIZE'], collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
testing_loader = torch.utils.data.DataLoader(ds['test'], batch_size=FLAGS['BATCH_SIZE'], collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))

device = xm.xla_device()

In [11]:
cnt = 0
for param in model.parameters():
    cnt += 1
    param.requires_grad = False
    if cnt > 270:
        param.requires_grad = True

config = AutoConfig.from_pretrained(MODEL)
num_devices = xr.global_runtime_device_count()
mesh_shape = (1, num_devices, 1)
device_ids = np.array(range(num_devices))
mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))
partition_module(model, mesh) # After this, the model is sharded between cores but still has the same API as if it was on single device. Neat.

In [12]:
!export XLA_USE_BF16=1

def train(FLAGS):
    num_iterations = int(FLAGS['NUM_STEPS'] / FLAGS['BATCH_SIZE'])
    lr = 1e-5
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=FLAGS['NUM_STEPS'] * FLAGS['BATCH_SIZE'])
    for epoch in range(1, FLAGS['NUM_EPOCHS'] + 1):
        model.train()
        xm.master_print('Epoch {} train begin {}'.format(epoch, test_utils.now()))
        for step, batch in enumerate(training_loader):
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch.input_ids.to(device), batch.attention_mask.to(device),  torch.unsqueeze(batch.labels.to(device), 1)
            xs.mark_sharding(input_ids, mesh, (0, 1)) # Sharding inputs
            xs.mark_sharding(attention_mask, mesh, (0, 1))
            xs.mark_sharding(labels, mesh, (0, 1))
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            xm.mark_step()
            if (step + 1) % FLAGS['LOGGING_STEPS'] == 0:
                print(f'loss: {loss.item()}, time: {test_utils.now()}, step: {step}')
            scheduler.step()

        model.eval()
        total_loss = 0.0
        total_steps = 0

        with torch.no_grad():
            for step, batch in enumerate(testing_loader):
                input_ids, attention_mask, labels = batch.input_ids.to(device), batch.attention_mask.to(device), torch.unsqueeze(batch.labels.to(device), 1)
                xs.mark_sharding(input_ids, mesh, (0, 1))
                xs.mark_sharding(attention_mask, mesh, (0, 1))
                xs.mark_sharding(labels, mesh, (0, 1))
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_loss += loss.item()
                total_steps += 1

        average_loss = total_loss / total_steps
        xm.master_print('Epoch {} test end {}, test loss={:.2f}'.format(epoch, test_utils.now(), average_loss))
        xm.master_print('Epoch {} train end {}'.format(epoch, test_utils.now()))

In [13]:
train(FLAGS) # I haven't tested the evaluation part in this notebook so hopefully it works. It really should

Epoch 1 train begin 15:17:49
loss: 0.5859375, time: 15:20:18, step: 99
loss: 0.75, time: 15:21:25, step: 199
loss: 0.23828125, time: 15:22:31, step: 299
loss: 0.52734375, time: 15:23:38, step: 399
loss: 0.9609375, time: 15:24:44, step: 499
loss: 0.341796875, time: 15:25:51, step: 599
loss: 0.240234375, time: 15:26:57, step: 699
loss: 0.283203125, time: 15:28:04, step: 799
loss: 0.228515625, time: 15:29:10, step: 899
loss: 0.37109375, time: 15:30:17, step: 999
loss: 0.4765625, time: 15:31:23, step: 1099
loss: 0.76171875, time: 15:32:30, step: 1199
loss: 0.384765625, time: 15:33:36, step: 1299
loss: 0.154296875, time: 15:34:43, step: 1399
loss: 0.46875, time: 15:35:49, step: 1499
loss: 0.09423828125, time: 15:36:56, step: 1599
loss: 0.1962890625, time: 15:38:02, step: 1699
loss: 0.322265625, time: 15:39:09, step: 1799
loss: 0.263671875, time: 15:40:15, step: 1899
loss: 0.361328125, time: 15:41:22, step: 1999
loss: 0.66796875, time: 15:42:28, step: 2099
loss: 0.416015625, time: 15:43:35, 

In [14]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_write") # Provide your own HF API token with write access
login(hf_token)

model = model.cpu()
print('now saving the model')
model.push_to_hub(
    "Defetya/example_repo_detectAI", 
    tokenizer=tokenizer,
    private=False,
    create_pr=False,
    max_shard_size="2GB", # Sharding isn't as important as before since hardware is better now but who cares anyway
    )# We have to push the model to HF since there is not enough memory on disk. Download weights from there

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
now saving the model



model-00011-of-00014.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/918M [00:00<?, ?B/s]


Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]



model-00012-of-00014.safetensors:   0%|          | 8.19k/1.90G [00:00<13:33:54, 39.0kB/s]
model-00011-of-00014.safetensors:   0%|          | 8.19k/1.90G [00:00<16:37:12, 31.8kB/s]



model-00013-of-00014.safetensors:   0%|          | 8.19k/1.90G [00:00<15:51:27, 33.3kB/s]

model-00012-of-00014.safetensors:   0%|          | 4.37M/1.90G [00:00<01:47, 17.7MB/s]   
model-00011-of-00014.safetensors:   0%|          | 2.67M/1.90G [00:00<03:19, 9.52MB/s]   



model-00013-of-00014.safetensors:   0%|          | 2.88M/1.90G [00:00<02:59, 10.6MB/s]   

model-00012-of-00014.safetensors:   0%|          | 7.49M/1.90G [00:00<01:23, 22.6MB/s]
model-00011-of-00014.safetensors:   0%|          | 7.24M/1.90G [00:00<01:27, 21.7MB/s]

model-00014-of-00014.safetensors:   1%|          | 7.32M/

CommitInfo(commit_url='https://huggingface.co/Defetya/example_repo_detectAI/commit/fcd59a5d74ccba9b9303a2ee38f03c1f6d8705ce', commit_message='Upload LlamaForSequenceClassification', commit_description='', oid='fcd59a5d74ccba9b9303a2ee38f03c1f6d8705ce', pr_url=None, pr_revision=None, pr_num=None)

If you're interested in fine-tuning quantazed Mistral on TPU for this job, I suggest you check [this](https://www.kaggle.com/code/sandiago21/llm-science-exam-llms-training-tpu) notebook by @sandiago21 (it's fine-tuned for previous llm competition but you can easily adapt it). There are also some interesting insights on how TPU works.